# Sprawdzenie kernela 

Na początek sprawdź czy silnik wykonawczy Twojego notatnika to PySpark. 
Mógłby on być po prostu interpreterem Pythona, jednak wówczas zmienne kontekstu musielibyśmy tworzyć samodzielnie.

Sprawdź, czy obiekt kontekstu jest dostępny. W przypadku *Spark SQL* jest to `SparkSession`

In [ ]:
spark

Dzięki powyższej informacji dowiedzieliśmy się nie tylko w jakim trybie został uruchomiony Spark obsługujący nasze polecenia, w jakiej jest wersji, ale także czy obsługuje funkcjonalność platformy Hive.

Dowiedz się także pod jakim użytkownikiem działamy w ramach tego notatnika.

In [ ]:
%%sh 
whoami

Czas na nasze właściwe zadania. 

W razie potrzeby korzystaj z https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html

# 20 Years of Games

**7. Zaczytaj do zmiennej gameInfosDF zawartość pliku ign.csv**

In [ ]:
username = "jankiewicz_krzysztof" # UWAGA! ustaw zmienną username na poprawną wartość

gameInfosDF=spark.read.\
    option("inferSchema", "true").\
    csv(f"/user/{username}/ign.csv", header=True).cache()

**8. Wyświetl schemat zmiennej gameInfosDF**

In [ ]:
gameInfosDF.printSchema()

Możesz także po prostu przyglądnąć się jej kolumnom

In [ ]:
gameInfosDF.columns

Zobaczmy też trzy pierwsze wiersze. Zróbmy to na kilka sposobów. 

* Na początek metoda `show()`

In [ ]:
gameInfosDF.limit(3).show()

Przetwarzane dane mogą być duże. Wyniki natomiast z reguły są znacznie mniejsze, to pozwala nam je (o ile znamy ich wielkość) przekonwertować do obiektów `pandas DataFrame` i dzięki temu przedstawić w przyjaźniejszej postaci.
* metoda `toPandas()`

In [ ]:
gameInfosDF.limit(3).toPandas()

Za pomocą parametru konfiguracyjnego `spark.sql.repl.eagerEval.enabled` naszego kontekstu, również możemy 
ułatwić sobie wgląd w zawartość naszych wyników. Warto także ustawić parametr aby kontrolować liczbę pobieranych w ten sposób wierszy (tak, w razie niedoszacowania wyniku)
* parametr `spark.sql.repl.eagerEval.enabled`

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 3)
gameInfosDF

Wykorzystuj powyższe, aby móc podglądać uzyskiwane wyniki

 
**9. Na początek coś prostego. 
Wyświetl trzy najlepiej ocenione gry wydane w roku 2016 na platformę PC.**

In [ ]:
from pyspark.sql.functions import col, lit
# tu wprowadź swoje rozwiazanie
gameInfosDF.where(col("platform")=="PC").\
    where(col("release_year")=="2016").\
    orderBy(col("score").desc()).\
    limit(3).select("title","score").show()



**10. Określ dla każdej oceny opisowej (score_phrase) minimalną i 
maksymalną ocenę liczbową. Wyniki posortuj
rosnąco pod względem minimalnej oceny liczbowej.**

In [ ]:
from pyspark.sql.functions import *
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)
# tu wprowadź swoje rozwiazanie
gameInfosDF.groupBy("score_phrase").\
    agg(
        max(col("score")).alias("max_score"),
        min(col("score")).alias("min_score")
    ).\
    select("score_phrase","max_score","min_score").\
    orderBy(col("min_score").asc()).\
    show()

**11. To może coś trudniejszego. Wyznacz liczbę oraz średnią ocenę gier wydawanych w poszczególnych latach
począwszy od roku 2000 na poszczególne platformy. Nie analizuj wszystkich platform – ogranicz je tylko do
tych, dla których liczba wszystkich recenzji gier biorąc pod uwagę wszystkie lata przekroczyła 500.**

*Uwaga: Klasycznie odwołalibyśmy się do źródłowego zboru danych dwa razy. Raz aby wyznaczyć popularne platformy, a następnie aby wyznaczyć ostateczny wynik. 
Korzystając z funkcji analitycznych możesz to zadanie rozwiązać sięgając do źródłowych danych tylko raz.*

**Rozwiąż to zadanie na dwa sposoby:**

a. Za pomocą DataFrame API


In [ ]:
from pyspark.sql.window import Window
# tu wprowadź swoje rozwiazanie


b. Za pomocą SQL (po zarejestrowaniu źródeł danych jako tymczasowych perspektyw).

In [ ]:
gameInfosDF.createOrReplaceTempView("gameinfos")
# tu wprowadź swoje rozwiazanie


**12. Jeśli masz swoją ulubioną serię gier (https://pl.wikipedia.org/wiki/Kategoria:Serie_gier_komputerowych)
zobacz jakie średnie oceny zdobyły poszczególne pozycje z tej serii. Wyniki posortuj chronologicznie.**

In [ ]:
# tu wprowadź swoje rozwiazanie


**13. (opcjonalne) Porównaj ze sobą gry wchodzące w skład wybranych serii gier wchodzących w skład 20
najlepszych serii wg Guinessa (lista z 2010 roku). W związku z tym, że gry nie są wydawane co roku, pogrupuj
dane w przedziały o długości 5 lat.**

In [ ]:
# tu wprowadź swoje rozwiazanie


In [ ]:
# brudnopis

 
# MondialDB – DataFrames

**14. Na początku do zmiennych `citiesDF`, `countriesDF` załaduj odpowiednio dane z plików
`mondial.cities.json`, `mondial.countries.json`**

In [ ]:
citiesDF = spark.read.json(f"/user/{username}/mondial.cities.json").cache()
countriesDF = spark.read.json(f"/user/{username}/mondial.countries.json").cache()

**15. Zapoznaj się z ich strukturą. Zwróć uwagę na występujące typy array.**

In [ ]:
citiesDF.printSchema()
countriesDF.printSchema()

**16. Zanim zaczniesz realizować zadania, zapoznaj się ze funkcją `explode`, która nadaje się świetnie do pracy z tablicami i ich rozpłaszczania.**

**Przykładowe zapytanie:**

In [ ]:
countriesDF.where("name = 'Poland'").\
            select(col("name"), explode(col("population")).alias("pop_in_years"))

Zwróć uwagę także na inne funkcje z tej rodziny jak: `explode_outer`, `posexplode`, `posexplode_outer`
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html


Wszystkie zadania wykonaj korzystając *DataFrame API*. Nie korzystaj z SQL.

**17. Oblicz sumę ludności wszystkich Państw na rok 2010. 
W sytuacji gdy w danym kraju nie przeprowadzono
badania w roku 2010 wykorzystaj najnowsze z badań wcześniejszych.**

In [ ]:
from pyspark.sql.window import Window
# tu wprowadź swoje rozwiazanie


**18. Było ciężko? Nie wierzę.**

**Teraz już będzie z górki. Podaj nazwy i gęstość zaludnienia trzech krajów o największej gęstości zaludnienia w roku 2010.**

In [ ]:
# tu wprowadź swoje rozwiazanie



**19. Podaj trzy kraje o największym procencie ludności żyjącym w miastach powyżej 50 000 mieszkańców w roku
2010.**

In [ ]:
# tu wprowadź swoje rozwiazanie


No cóż, dane dotyczące ludności w miastach są zapewne nowsze niż z 2010 roku.
Na marginesie, zarówno Melilla jak i Ceuta to hiszpańskie miasta, afrykańskie eksklawy położone na terytorium
Maroka. Oba liczą ponad 70 tyś mieszkańców i oba posiadają autonomię (uzyskaną jednocześnie w marcu 1995 roku)
dlatego znalazły się w naszym zestawieniu.
A co to takiego eksklawy i czy enklawa jest tym samym, to już możesz przeczytać samodzielnie np. tu:
https://pl.wikipedia.org/wiki/Eksklawa

In [ ]:
# brudnopis